In [2]:
from utils import * 
from notes import NOTES 
from src.files.blast import BLASTFileGgKbase
import glob
import parasail 
import math
import pyfaidx 

import networkx as nx
from tqdm import tqdm 
%load_ext autoreload 
%autoreload 2

# Nantong_Groundwater_SRR22387873_scaffold_155357 is too low converage and has too many SNPs, not doable. 
# Nantong_Groundwater_SRR22387873_scaffold_346753

# makeblastdb -in contigs.fa -dbtype nucl -title contigs

In [3]:
CONTIGS_PATH = '../data/databases/nantong_groundwater/contigs.fa'

def fai_get_sequences(contig_ids, path:str=CONTIGS_PATH, strands:list=None):
    '''Read the sequences from the FASTA file.'''
    f = pyfaidx.Fasta(path)
    strands = np.array(['+'] * len(contig_ids)) if (strands is None) else strands
    seqs = [str(f[contig_id]) for contig_id in contig_ids]
    seqs = [reverse_complement(seq) if (strand == '-') else seq for (strand, seq) in zip(strands, seqs)]
    return seqs

In [4]:
def make_genome(input_dir:str, output_dir:str='../data/'):
    contig_id_map = dict()
    genome_id = os.path.basename(input_dir)
    content = ''
    for i, path in enumerate(sorted(glob.glob(os.path.join(input_dir, '*')))):
        with open(path, 'r') as f:
            lines = f.readlines()
            contig_id_map[f'{genome_id}.{i + 1}'] = lines[0].split()[0].replace('>', '')
            lines = [f'>{genome_id}.{i + 1}\n'] + lines[1:]
            content += ''.join(lines)

    output_path = os.path.join(output_dir, f'{genome_id}.genome.fasta')
    with open(output_path, 'w') as f:
        f.write(content)
    return output_path


In [5]:
# Nantong_Groundwater_SRR22387873_scaffold_1715 is definitely bacterial. 
# Nantong_Groundwater_SRR22387873_scaffold_2044 is already binned as an ECE. 
# On Nantong_Groundwater_SRR22387873_scaffold_239429, the second gene bears resemblance to the DNA polymerase.


def load_database(path:str='../data/databases/nantong_groundwater/taxonomy.tsv'):
    # Contig name	Size (bp)	Coverage	GC %	Taxonomy winner	Winner %	Species winner	Species winner %	Genus winner	Genus winner %	Order winner	Order winner %	Class winner	Class winner %	Phylum winner	Phylum winner %	Domain winner	Domain winner %
    levels = ['species', 'genus', 'order', 'class', 'phylum', 'domain']
    cols = ['contig_id', 'size', 'coverage', 'gc_percent', 'taxonomy_winner', 'taxonomy_winner_percent']
    cols = cols + [col for level in levels for col in [f'{level}_winner', f'{level}_winner_percent']]
    database_df = pd.read_csv(path, sep='\t', names=cols, skiprows=1)
    database_df = database_df.fillna('unknown')
    return database_df.set_index('contig_id')

database_df = load_database()

In [13]:
seed_contigs_df = pd.read_csv('../data/seed_contigs.csv', index_col=0)

seed_contig_id = 'Nantong_Groundwater_SRR22387873_scaffold_346753'
query_row = seed_contigs_df.loc[seed_contig_id]
print(seed_contig_id)
print(f'GC percent: {100 * query_row.gc_content:.2f}%')
print(f'Length: {query_row.length} bp')

Nantong_Groundwater_SRR22387873_scaffold_346753
GC percent: 22.06%
Length: 1047 bp


In [14]:
def search_database(contig_id, database_df=database_df, gc_percent_delta:float=3, coverage_delta:float=2, notes=NOTES, exclude_flagged:bool=True):
    '''Search the database_df for contigs with similar coverage and GC content to the seed contig. '''
    query_row = database_df.loc[contig_id]
    mask = (database_df.gc_percent < query_row.gc_percent + gc_percent_delta) & (database_df.gc_percent > query_row.gc_percent - gc_percent_delta)
    mask = mask & ((database_df.coverage < query_row.coverage + coverage_delta) & (database_df.coverage > query_row.coverage - coverage_delta))
    
    results_df = database_df[mask].copy()
    results_df['note'] = results_df.index.map(notes)
    results_df['note'] = results_df.note.fillna('none')
    if exclude_flagged:
        results_df = results_df[~results_df.note.str.contains('X')].copy()
    results_df = results_df[results_df.domain_winner != 'Bacteria'].copy() # Exclude hits with Bacterial taxonomy winner.
    results_df = results_df.sort_values('size', ascending=False)
    results_df = results_df.loc[[contig_id] + [id_ for id_ in results_df.index if (id_ != contig_id)]].copy()
    print(f'search_database: Found {len(results_df)} with similar GC percent and coverage to {contig_id}.')
    print(f"search_database: {results_df['size'].sum() / 1000:.2f} kb in candidate bin.")
    return results_df.copy()


results_df = search_database(seed_contig_id)
results_df['seq'] = fai_get_sequences(results_df.index.values)

search_database: Found 12 with similar GC percent and coverage to Nantong_Groundwater_SRR22387873_scaffold_346753.
search_database: 26.46 kb in candidate bin.


In [15]:
CONTIG_END_LENGTH = 200
DATABASE_PATH = '/home/prichter/Documents/banfield/betazoid/data/databases/nantong_groundwater/contigs.fa'
OUTPUT_DIR = '../data/bins/nantong_groundwater/'
MIN_PERCENT_IDENTITY = 80
MIN_ALIGNMENT_LENGTH = 80


def _filter_hits(blast_df:pd.DataFrame, delta:int=3):
    mask = (blast_df.query_border != blast_df.subject_border) # Get rid of overlaps between left borders and right borders. 
    mask = mask & (((blast_df.query_border == 'L') & (blast_df.query_lbd < delta)) | ((blast_df.query_border == 'R') & (blast_df.query_rbd < delta)))
    mask = mask & (((blast_df.subject_border == 'L') & (blast_df.subject_lbd < delta)) | ((blast_df.subject_border == 'R') & (blast_df.subject_rbd < delta)))
    print(f'_filter_hits: Retaining {mask.sum()} out of {len(blast_df)} hits.')
    blast_df = blast_df[mask].copy()
    return blast_df

# Need some functions to account for merged contigs. 
get_scaffold_numbers = lambda contig_id : contig_id.split('_scaffold_')[-1].split('_')
# has_shared_scaffold = lambda contig_id_1, contig_id_2 : len(np.intersect1d(get_scaffold_numbers(contig_id_1), get_scaffold_numbers(contig_id_2)))
has_shared_scaffold = lambda row : len(np.intersect1d(get_scaffold_numbers(row.query_id), get_scaffold_numbers(row.subject_id))) > 0

def blast_contig_ends(contigs_df:pd.DataFrame, name:str=None, max_n_mismatches:int=2):

    input_path = os.path.join(OUTPUT_DIR, f'{name}.blast.in.fasta')
    output_path = os.path.join(OUTPUT_DIR, f'{name}.blast.out.tsv')
    input_df = [{'seq':row.seq[:CONTIG_END_LENGTH], 'id':f'{row.Index}.L'} for row in contigs_df.itertuples()]
    input_df += [{'seq':row.seq[-CONTIG_END_LENGTH:], 'id':f'{row.Index}.R'} for row in contigs_df.itertuples()]
    input_df = pd.DataFrame(input_df).set_index('id')
    FASTAFile.from_df(input_df).write(input_path)

    fields = 'qseqid sseqid qlen slen length mismatch gapopen qstart qend sstart send' #.split()
    cmd = f'blastn -db {DATABASE_PATH} -query {input_path} -out {output_path} -perc_identity {MIN_PERCENT_IDENTITY} -outfmt "6 {fields}"'

    subprocess.run(cmd, shell=True, check=True)
    blast_df = pd.read_csv(output_path, sep='\t', names=fields.split())
    blast_df = blast_df[blast_df.mismatch <= max_n_mismatches].copy()
    # blast_df = blast_df[blast_df.qseqid.str.partition(".")[0] != blast_df.sseqid].copy() # Remove self-alignments.
    blast_df = blast_df[blast_df.length > MIN_ALIGNMENT_LENGTH].copy() 

    # First load the subject sequences and clean up. 
    # Convert from one-indexed inclusive bounds to zero-indexed, upper-exclusive bounds. 

    blast_df['subject_id'] = blast_df.sseqid
    blast_df['subject_strand'] = np.where(blast_df.send < blast_df.sstart, '-', '+')
    blast_df['subject_length'] = blast_df.slen
    blast_df['subject_seq'] = fai_get_sequences(blast_df.subject_id, strands=blast_df.subject_strand.values)
    blast_df['subject_start'] = np.where(blast_df.subject_strand == '+', blast_df.sstart - 1, blast_df.slen - blast_df.sstart)
    blast_df['subject_end'] = np.where(blast_df.subject_strand == '+', blast_df.send, blast_df.slen - blast_df.send + 1)
    blast_df['subject_rbd'] = blast_df.subject_length - blast_df.subject_end
    blast_df['subject_lbd'] = blast_df.subject_start
    blast_df['subject_border'] = np.where(blast_df.subject_lbd < blast_df.subject_rbd, 'L', 'R')

    blast_df['query_id'] = blast_df.qseqid.str.partition(".")[0]
    blast_df['query_seq'] = blast_df.query_id.map(contigs_df.seq)
    blast_df['query_length'] = blast_df.query_seq.apply(len)
    blast_df['query_border'] = [re.search(r'\.(R|L)', id_).group(1) for id_ in blast_df.qseqid]
    blast_df['query_start'] = np.where(blast_df.query_border == 'R', (blast_df.query_length - CONTIG_END_LENGTH) + (blast_df.qstart - 1), blast_df.qstart - 1) # If query match is on the right border, the start of the alignment is (150 - qstart) from the right end. 
    blast_df['query_end'] = np.where((blast_df.query_border == 'R'),  (blast_df.query_length - CONTIG_END_LENGTH) + blast_df.qend, blast_df.qend) # If query match is on the right border, use the end of the alignment is (150 - qend) from the right end. 
    blast_df['query_rbd'] = blast_df.query_length - blast_df.query_end
    blast_df['query_lbd'] = blast_df.query_start

    blast_df = blast_df[~blast_df.apply(has_shared_scaffold, axis=1)].copy() # Remove self-hits.

    blast_df = blast_df.rename(columns={'gapopen':'n_gaps', 'mismatch':'n_mismatches', 'length':'alignment_length'})
    cols = ['query_id', 'subject_id', 'query_length', 'subject_length', 'subject_strand'] 
    cols += ['query_border', 'query_lbd', 'query_rbd', 'subject_border', 'subject_lbd', 'subject_rbd']
    cols += ['query_start', 'query_end', 'subject_start', 'subject_end']
    cols += ['n_gaps', 'n_mismatches', 'alignment_length', 'query_seq', 'subject_seq']
    return blast_df[cols].copy().reset_index()


In [ ]:
def merge_contigs(blast_df:pd.DataFrame, database_df:pd.DataFrame=database_df):
    get_scaffold_number = lambda contig_id : contig_id.split('_scaffold_')[-1] #.split('_')
    get_scaffold_prefix = lambda contig_id : contig_id.split('_scaffold_')[0]

    df = list()
    for row in blast_df.itertuples():
        row_ = {'parent_contig_ids':[row.query_id, row.subject_id], 'parent_lengths':[row.query_length, row.subject_length], 'alignment_length':row.alignment_length}
        row_['parent_coverages'] = database_df.loc[[row.query_id, row.subject_id]].coverage.tolist()
        row_['parent_gc_percents'] = database_df.loc[[row.query_id, row.subject_id]].gc_percent.tolist()

        if row.query_border == 'R':
            row_['seq'] = row.query_seq + row.subject_seq[row.subject_end:]
            row_['contig_id'] = get_scaffold_prefix(row.query_id) + '_scaffold_' + get_scaffold_number(row.query_id) + '_' + get_scaffold_number(row.subject_id)
        elif row.query_border == 'L':
            row_['seq'] = row.subject_seq[:row.subject_start] + row.query_seq
            row_['contig_id'] = get_scaffold_prefix(row.query_id) + '_scaffold_' + get_scaffold_number(row.subject_id) + '_' + get_scaffold_number(row.query_id)
        
        L = sum(row_['parent_lengths'])
        row_['coverage'] = np.mean([(l /L)  * gc for l, gc in zip(row_['parent_lengths'], row_['parent_gc_percents'])]) # This will not be exact because of the overlap.
        row_['gc_percent'] = 100 * (row_['seq'].count('G') + row_['seq'].count('C')) / len(row_['seq'])
        df.append(row_)
    df = pd.DataFrame(df)
    df['length'] = df.seq.apply(len)
    
    # print(df.query_length + df.subject_length - df.alignment_length)
    return df.set_index('contig_id', drop=True)


In [ ]:
blast_df = blast_contig_ends(results_df, name='scaffold_346753')


In [18]:
merge_contigs(blast_df)

,parent_contig_ids,parent_lengths,alignment_length,parent_coverages,parent_gc_percents,seq,coverage,gc_percent,length
contig_id,,,,,,,,,
Nantong_Groundwater_SRR22387873_scaffold_54325_6662,[Nantong_Groundwater_SRR22387873_scaffold_6662...,"[7807, 2557]",99,"[25.0, 17.01]","[23.07, 25.54]",TGACAGATTATGAGGAGTATGTAAAATGAAAATAGAAGAAATGATT...,11.839698,23.633707,10265
Nantong_Groundwater_SRR22387873_scaffold_76869_17251,[Nantong_Groundwater_SRR22387873_scaffold_1725...,"[4630, 2152]",99,"[22.42, 10.94]","[24.95, 27.97]",AGATTTGGGATATAAAAAATGAAAATATTATTACTTAGATTAGATT...,12.954139,25.976358,6683
Nantong_Groundwater_SRR22387873_scaffold_25391_17251,[Nantong_Groundwater_SRR22387873_scaffold_1725...,"[4630, 3770]",99,"[22.42, 15.32]","[24.95, 26.63]",TCAAGAAATGGACTTCCCTTATGGAATAAAAATAGGTAAAGAAGGA...,12.852000,25.755933,8301
Nantong_Groundwater_SRR22387873_scaffold_76591_78449,[Nantong_Groundwater_SRR22387873_scaffold_7844...,"[2131, 2156]",99,"[24.28, 13.36]","[24.82, 29.08]",ATTTGCAGGATTGACAGAAAAATCATCAAATGTGGGAAGTGCGAAA...,13.481211,27.005731,4188
Nantong_Groundwater_SRR22387873_scaffold_49642_78449,[Nantong_Groundwater_SRR22387873_scaffold_7844...,"[2131, 2674]",99,"[24.28, 14.92]","[24.82, 30.1]",CCCATGTATTGAACGAAAATGCCAAGCCCTGCTCCTGTTCCGCTTA...,13.879170,27.815555,4706
Nantong_Groundwater_SRR22387873_scaffold_370795_194246,[Nantong_Groundwater_SRR22387873_scaffold_1942...,"[1376, 1015]",99,"[21.58, 9.01]","[24.2, 25.81]",GGGCATTTATCTTATCCTTTCTATTTCTTTTAATAAATCTTCAAAT...,12.441729,24.694590,2292
Nantong_Groundwater_SRR22387873_scaffold_193169_194246,[Nantong_Groundwater_SRR22387873_scaffold_1942...,"[1376, 1380]",99,"[21.58, 6.52]","[24.2, 25.8]",GGGCATTTATCTTATCCTTTCTATTTCTTTTAATAAATCTTCAAAT...,12.500581,24.840045,2657
Nantong_Groundwater_SRR22387873_scaffold_37915_267693,[Nantong_Groundwater_SRR22387873_scaffold_2676...,"[1182, 3066]",112,"[22.08, 20.84]","[24.2, 38.62]",TTTTCATGTTAATAATACAAGTGGAAGCACTTTAACCGTTGGCAAT...,17.303828,35.193026,4015
Nantong_Groundwater_SRR22387873_scaffold_17251_78447,[Nantong_Groundwater_SRR22387873_scaffold_1725...,"[4630, 2131]",99,"[22.42, 30.27]","[24.95, 27.31]",TTTTATAAAAAAACATTAAATATAAAATATTATTTTACAGGAGAAA...,12.846924,25.712999,6662


In [19]:
blast_df

,index,query_id,subject_id,query_length,subject_length,subject_strand,query_border,query_lbd,query_rbd,subject_border,...,subject_rbd,query_start,query_end,subject_start,subject_end,n_gaps,n_mismatches,alignment_length,query_seq,subject_seq
0,3,Nantong_Groundwater_SRR22387873_scaffold_6662,Nantong_Groundwater_SRR22387873_scaffold_54325,7807,2557,-,L,0,7708,R,...,0,0,99,2458,2557,0,0,99,AAAGATAATTTGAAATCACTTTGTTGCAGAGCCAAAGTTAAAATAG...,TGACAGATTATGAGGAGTATGTAAAATGAAAATAGAAGAAATGATT...
1,5,Nantong_Groundwater_SRR22387873_scaffold_17251,Nantong_Groundwater_SRR22387873_scaffold_76869,4630,2152,+,L,0,4531,R,...,0,0,99,2053,2152,0,0,99,TTTTATAAAAAAACATTAAATATAAAATATTATTTTACAGGAGAAA...,AGATTTGGGATATAAAAAATGAAAATATTATTACTTAGATTAGATT...
2,6,Nantong_Groundwater_SRR22387873_scaffold_17251,Nantong_Groundwater_SRR22387873_scaffold_25391,4630,3770,+,L,0,4531,R,...,0,0,99,3671,3770,0,0,99,TTTTATAAAAAAACATTAAATATAAAATATTATTTTACAGGAGAAA...,TCAAGAAATGGACTTCCCTTATGGAATAAAAATAGGTAAAGAAGGA...
3,11,Nantong_Groundwater_SRR22387873_scaffold_78449,Nantong_Groundwater_SRR22387873_scaffold_76591,2131,2156,+,L,0,2032,R,...,0,0,99,2057,2156,0,0,99,GCTATTTCTATTACAATATTTGAAAGCTCTTTGGCAATTTCATCAA...,ATTTGCAGGATTGACAGAAAAATCATCAAATGTGGGAAGTGCGAAA...
4,12,Nantong_Groundwater_SRR22387873_scaffold_78449,Nantong_Groundwater_SRR22387873_scaffold_49642,2131,2674,+,L,0,2032,R,...,0,0,99,2575,2674,0,0,99,GCTATTTCTATTACAATATTTGAAAGCTCTTTGGCAATTTCATCAA...,CCCATGTATTGAACGAAAATGCCAAGCCCTGCTCCTGTTCCGCTTA...
5,14,Nantong_Groundwater_SRR22387873_scaffold_194246,Nantong_Groundwater_SRR22387873_scaffold_370795,1376,1015,+,L,0,1277,R,...,0,0,99,916,1015,0,0,99,CTACTGCAATTGGTTCAAATACGAAACCATAGGGTCTTAATTTTTT...,GGGCATTTATCTTATCCTTTCTATTTCTTTTAATAAATCTTCAAAT...
6,15,Nantong_Groundwater_SRR22387873_scaffold_194246,Nantong_Groundwater_SRR22387873_scaffold_193169,1376,1380,-,L,0,1277,R,...,0,0,99,1281,1380,0,0,99,CTACTGCAATTGGTTCAAATACGAAACCATAGGGTCTTAATTTTTT...,GGGCATTTATCTTATCCTTTCTATTTCTTTTAATAAATCTTCAAAT...
7,20,Nantong_Groundwater_SRR22387873_scaffold_267693,Nantong_Groundwater_SRR22387873_scaffold_37915,1182,3066,+,L,0,1070,R,...,122,0,112,2833,2944,1,2,112,ATAAACCAATATCAGAACTAACTCAGCAAAATACTATTGGTATGCA...,TTTTCATGTTAATAATACAAGTGGAAGCACTTTAACCGTTGGCAAT...
8,33,Nantong_Groundwater_SRR22387873_scaffold_17251,Nantong_Groundwater_SRR22387873_scaffold_78447,4630,2131,-,R,4531,0,L,...,2032,4531,4630,0,99,0,0,99,TTTTATAAAAAAACATTAAATATAAAATATTATTTTACAGGAGAAA...,AAAGTAGAAAAAATTTATGATGAAGATTTAGATATTAGATTGGAAA...
9,37,Nantong_Groundwater_SRR22387873_scaffold_78449,Nantong_Groundwater_SRR22387873_scaffold_263905,2131,1190,+,R,2032,0,L,...,1091,2032,2131,0,99,0,0,99,GCTATTTCTATTACAATATTTGAAAGCTCTTTGGCAATTTCATCAA...,CTGGAATATATATTTTATTGTTCTCGCAGATGGTAAAAAGAAGCTC...


In [ ]:
# replace_contig_ids = dict()
# replace_contig_ids['Nantong_Groundwater_SRR22387873_scaffold_377775'] = 'Nantong_Groundwater_SRR22387873_scaffold_103075_377775'
# replace_contig_ids['Nantong_Groundwater_SRR22387873_scaffold_358410'] = 'Nantong_Groundwater_SRR22387873_scaffold_153884_358410'
# replace_contig_ids['Nantong_Groundwater_SRR22387873_scaffold_335499'] = 'Nantong_Groundwater_SRR22387873_scaffold_45379_335499_207268'

# for contig_id, new_contig_id in replace_contig_ids.items():
#     results_df.loc[contig_id, 'size'] = new_contigs_df.loc[new_contig_id, 'length']
#     results_df.loc[contig_id, 'gc_percent'] = new_contigs_df.loc[new_contig_id, 'gc_percent']
#     results_df.loc[contig_id, 'coverage'] = new_contigs_df.loc[new_contig_id, 'coverage']
#     results_df.loc[contig_id, 'seq'] = new_contigs_df.loc[new_contig_id, 'seq']
#     results_df.rename(index={contig_id:new_contig_id}, inplace=True)

# FASTAFile.from_df(results_df).write(f"../data/bins/nantong_groundwater/{query_row['id']}.ref.fasta")


KeyError: 'Nantong_Groundwater_SRR22387873_scaffold_103075_377775'

In [ ]:
results_df.sort_values('size')
results_df['size'].sum()

np.int64(51543)

In [ ]:
print(database_df.loc['Nantong_Groundwater_SRR22387873_scaffold_45379']['size'])
print(database_df.loc['Nantong_Groundwater_SRR22387873_scaffold_335499']['size'] + database_df.loc['Nantong_Groundwater_SRR22387873_scaffold_45379']['size'])

print(database_df.loc['Nantong_Groundwater_SRR22387873_scaffold_103075']['size'])


2797
3860
1865


In [ ]:
def align_contig_ends(contigs_df, min_percent_identity:float=0.98, min_alignment_length:int=15, length:int=150, gap_open:int=20, gap_extend:int=1):
    contigs_df['reverse_seq'] = [reverse_complement(seq) for seq in contigs_df.seq] 
    contigs_df['seq_l'] = [seq[:length] for seq in contigs_df.seq]
    contigs_df['seq_r'] = [seq[-length:] for seq in contigs_df.seq]
    contigs_df['reverse_seq_l'] = [seq[:length] for seq in contigs_df.reverse_seq]
    contigs_df['reverse_seq_r'] = [seq[-length:] for seq in contigs_df.reverse_seq]

    matrix = parasail.matrix_create('ACGT', 2, -3) # + 2 match score, -3 mismatch score. 
    graph = nx.DiGraph()

    rows = list(contigs_df.itertuples())
    for a, b in tqdm(itertools.product(rows, rows), desc='align_contig_ends'):
        if a.Index == b.Index:
            continue 

        for loc_a, loc_b in [('seq_r', 'seq_l'), ('seq_r', 'reverse_seq_l'), ('reverse_seq_r', 'seq_l')]:
            alignment = parasail.sw_trace_striped_16(getattr(a, loc_a), getattr(b, loc_b), gap_open, gap_extend, matrix) # Smith-Waterman local alignment. 
            n_matches = sum(x == y for x, y in zip(alignment.traceback.query, alignment.traceback.ref) if ((x != '-') and (y != '-')))
            n = sum(1 for x, y in zip(alignment.traceback.query, alignment.traceback.ref) if ((x != '-') and (y != '-')))
            # print(alignment.traceback.comp)
            if ((n_matches / n) > min_percent_identity) and (n > min_alignment_length):
                graph.add_edge(a.Index, b.Index, percent_identity=(n_matches / n), alignment_length=n, alignment=f'{alignment.traceback.query}\n{alignment.traceback.ref}')
                
    return graph 

In [ ]:
graph = align_contig_ends(results_df)
# fig, ax = plt.subplots(figsize=(6, 6))
# pos = nx.spring_layout(graph)
# nx.draw_networkx_nodes(graph, pos, ax=ax, node_size=100)
# nx.draw_networkx_edges(graph, pos, ax=ax, width=0.5)

# for _, spine in ax.spines.items():
#     spine.set_visible(False)
# plt.show()

align_contig_ends: 961it [00:00, 6886.32it/s]


In [ ]:
for u, v, data in graph.edges(data=True):
    print(u, v)
    print(data['alignment'])
    print()

Nantong_Groundwater_SRR22387873_scaffold_155983 Nantong_Groundwater_SRR22387873_scaffold_315512
TATTCTATTTTATTTA
TATTCTATTTTATTTA



In [ ]:
results_df['size'].sum()
len(results_df)

31

In [ ]:
results_df.sort_values('size')


,size,coverage,gc_percent,taxonomy_winner,taxonomy_winner_percent,species_winner,species_winner_percent,genus_winner,genus_winner_percent,order_winner,...,phylum_winner_percent,domain_winner,domain_winner_percent,note,seq,reverse_seq,seq_l,seq_r,reverse_seq_l,reverse_seq_r
contig_id,,,,,,,,,,,,,,,,,,,,,
Nantong_Groundwater_SRR22387873_scaffold_377775,1006,15.66,24.85,unknown,unknown,unknown,1.0,unknown,1.0,unknown,...,1.0,unknown,1.0,none,TCTTTGTAGTTGAGATAATGAAATTCTGTCTTTTTTTCTTTTCATA...,TTCTTTATTGTAATACGAAATTAAAATGGCATATAAAGTAAAAAAA...,TCTTTGTAGTTGAGATAATGAAATTCTGTCTTTTTTTCTTTTCATA...,ATTATTGCTTTTTTTTTCTTGCTGATTTTATGAAAAGTGAATCCTT...,TTCTTTATTGTAATACGAAATTAAAATGGCATATAAAGTAAAAAAA...,TGAAAGAATTAGGAATTAAATATAAATTATCAAAATAAAATAATGA...
Nantong_Groundwater_SRR22387873_scaffold_371531,1014,15.24,25.05,unknown,unknown,unknown,1.0,unknown,1.0,unknown,...,1.0,unknown,1.0,none,AAATGGAAGAAGAAATACAAGAACTTTTTAAAACTATGGCAGAACA...,TAACTGTTGCATCTGGGACAACATTTCGGATCTTCTTTAATGCTTT...,AAATGGAAGAAGAAATACAAGAACTTTTTAAAACTATGGCAGAACA...,ATAAAAAAGGACATTTTATTTGGAATGAAAATAAAATCTCAAAATT...,TAACTGTTGCATCTGGGACAACATTTCGGATCTTCTTTAATGCTTT...,ATCTTTAAATATTATTTCTTTCATTTTTCAATTCCTCAATTTCTTT...
Nantong_Groundwater_SRR22387873_scaffold_369987,1016,15.50,25.30,Borg,0.67,FINAL_Emerald_Borg_35_17_with_NANOPORE_100cm-2...,0.33,Emerald,0.33,unknown,...,0.67,ECE,0.67,"Has two hypothetical Borg proteins, one Orange...",TATGACAAACTCTCCAAAAATGAAATGTCCTTTGAAGAAAAATACA...,TTTCTGTTGCTCTAATATCCATTTTTTGCTATATTCTTTAACTTTA...,TATGACAAACTCTCCAAAAATGAAATGTCCTTTGAAGAAAAATACA...,TATCAGAAAAAGAAAACATTTACAATAAAATAAATAATTTTTATGA...,TTTCTGTTGCTCTAATATCCATTTTTTGCTATATTCTTTAACTTTA...,TAAAGAAATTGCATTGTCTAATTCATATGGCAAACTAAAATAATCT...
Nantong_Groundwater_SRR22387873_scaffold_363017,1025,15.66,20.88,unknown,unknown,unknown,1.0,unknown,1.0,unknown,...,1.0,unknown,1.0,none,GAAAAACAACCTATTTATTACAGGTGCAAAGAACAGGTGTAAGGTG...,GATTTTTAAAATAATTGATAGAAGAGAAAATATAATACAAAAACAA...,GAAAAACAACCTATTTATTACAGGTGCAAAGAACAGGTGTAAGGTG...,GCTATTTATTTATTCTTTTATTTTATATATGAAGGAATTAAAGATT...,GATTTTTAAAATAATTGATAGAAGAGAAAATATAATACAAAAACAA...,TAAGGAGCACATCCAAGTTTTATATTATGCACAATATCACCCTTTT...
Nantong_Groundwater_SRR22387873_scaffold_358410,1031,16.44,25.61,unknown,unknown,unknown,1.0,unknown,1.0,unknown,...,1.0,unknown,1.0,none,AAGAATTATCTAATAGTTTGCATTTAATATGATATAACTAATCCCA...,GATAAATAATATAATATGCTCATAGGAATTTTCGGTAATGATTTTG...,AAGAATTATCTAATAGTTTGCATTTAATATGATATAACTAATCCCA...,AAATTTGAGGGCATAATTTGAATACACTTTCATATTATTCCTATAG...,GATAAATAATATAATATGCTCATAGGAATTTTCGGTAATGATTTTG...,AAAAATTAAAAGCTGAATTGGAAGGTTTTTGAGGAGTATTTTGCAA...
Nantong_Groundwater_SRR22387873_scaffold_335499,1063,13.69,25.12,unknown,unknown,unknown,0.8,unknown,0.8,unknown,...,0.8,unknown,0.8,Has a hypothetical Emerald Borg protein.,ACATTCTACACAAAAATCTGGTAATGAACAATCATTTATAGTATCC...,ATTATGTAATTTTATTATTGGTAAGGATAATAAATTTACTTACTAT...,ACATTCTACACAAAAATCTGGTAATGAACAATCATTTATAGTATCC...,TTCTCTTAATACTCCATCAATATCAAATGCTATTTTCATCTGAAAT...,ATTATGTAATTTTATTATTGGTAAGGATAATAAATTTACTTACTAT...,TAGTGTTGTTACAGATTTATATAATGGGCAAGAAAATATAGAACTT...
Nantong_Groundwater_SRR22387873_scaffold_329778,1072,14.97,25.65,unknown,unknown,unknown,1.0,unknown,1.0,unknown,...,1.0,unknown,1.0,none,TGTGATTTAGAAAGTACAGAATTATGTCCATATAATATAATGATAA...,AAAACTGGTGAAATATTTTACATTAAGGATACAGAATTTAGTTTAC...,TGTGATTTAGAAAGTACAGAATTATGTCCATATAATATAATGATAA...,TTGTGCTTCGTTTATAGTCATTAATTGACAGTCAAAATTTTCGAAA...,AAAACTGGTGAAATATTTTACATTAAGGATACAGAATTTAGTTTAC...,TTCTACCAATCTATCAATATCTTTTTCTTTTAATGTATTAATATTA...
Nantong_Groundwater_SRR22387873_scaffold_315512,1094,15.08,25.78,unknown,unknown,unknown,1.0,unknown,1.0,unknown,...,1.0,unknown,1.0,none,AGCACCACTAAGAAATCGTTAAATTTCTTAAATGGTGCTCTACGGC...,GAATATTAAAATCTGTAATAGAAAACTTTTTATAGGTTCGCCAGGG...,AGCACCACTAAGAAATCGTTAAATTTCTTAAATGGTGCTCTACGGC...,ACCCTCACAATTACACTTTCCTTAATCTCACAACTAACTATATTTA...,GAATATTAAAATCTGTAATAGAAAACTTTTTATAGGTTCGCCAGGG...,AAATTTTAAATAAAGATTTTTACAAAAGAAATTATTTAGCCAAACT...
Nantong_Groundwater_SRR22387873_scaffold_310196,1103,15.10,21.67,unknown

In [ ]:
s = '''AAAAACAGATTCTACCTATATAATAGGGTAGAAATGAGGTAGGAGTAAAAGTAGATAGCT
AACATATTACTATTCAATAAGTTAAATAAGGTAGAAACTTATACTTCTACCTTTGTATTA
CTTTTGTAATACCTATTCTTATATATTTAAAAAAATAAGAAGATAGGCTACTCATTTATA
TAGTATCTTTGCTAAAAAGAACATACCTCCGACCAGGCTCCGAATAGGGCATTTTTGACC
ACCGAAGGATTATATCATGTTTTTATGTAAGTCTAATAAAATACGAATGTTATGATAAAA
AATAGATAAATTAATATTCTCCGAAGAGGCATATTTAATGAAAAACAAATTAGTGGGGCG
AAAAAAATATAACTTATTGATAATAAAGGAGATAAATTGGCGAAATCGATAGAAAGAAAT
AGTCAGCTATCTCCCGATATAGATATAGATTTTTGCTAAAATCAACCCGACCCGATTTAT
TATAACTTCTTTATTTGTAATAAGTTATGCTTATTTTATCTCATTTTATTTTATAATTTA
GTTAGAAACTAAATAAGAATTTAAAAGCATTTTTTAGGCATATTTTTAGCGCTTTGCTTT
TTATCTACCATTACCTAATTATAAAGACTTATTTAATAAGATTTGTTAAGGTTTATTTAT
TTTGTTTTTGACCTTGACATTAGGCAATTTTTATGTTATACTTTAGGTATGATTGAGAGA
TTAAAGACCTTAATAAAAAAGATAAGACAAAGTAAAGAGCCAGAAAGTTTATTTTATCAA
GTGTATTATAAAAGAGATTATAATAAATTTATTTTATTAAGTAGAGGATTATAAAATGTT
AAGAATAGGTGATAAAATTATTTTTGATAGGACTTTTAATGGACATAAGTTTATTGATAC
ATTTGAATTGATTTTTATAGATACTCTTTTAGATTTTACTTGTGATGTATGCGGGAGAAG
ATACCCAGAAAAAAAATTTATGTTTAAGTCAATTAATTCTGATGATATTCTTTATATAGG
AACTACTTGTATTAAGAAAGTAAAGGATAATAGAATAACTTATAGTATAAATACAGATAA
GAATAAAAGGCAATTAAATGATTAAAGTATTTATCCCGACAGATAAAGCACAAGGCAAGA
CTAATGTAAGAGGTTTATGGATAAATGATGAGGGCAAACTATTTTATGATTATTTAAGGC
AAGATGTTATTAATACTTTTTTTAATAGACAAAGGCATAAAATAATGTTAAAAAAATTAG
ATGATATAAGATTAAAATATAATCAAGAGGCATTATTTTTTATTGATAAGACTAATGATT
ATGATGTTGGATATATACTTAATAACAATTTAACAATAACAGAATTAAATAATGTTATAA
GATTTAACATAGGCAAGAATAAAAAGAATTTAAGAGGTATAATTAAAAAGTTATTAAGAG
ATTATAAAGGTTTAACTATTTATATAGAGCCAGAGGGCTACACTTTAGAAGTATATTATA
ATAAAGAGATTAAATAATGATAAAAATAAAATTATACCATTATAGTAATACAGACATTAA
AGATAAAATAAAAGTTAGTTATTATGGTTTAAATTACTTTACTTTTAATGATAGTCAGAT
TACTACGGTTAAGAGGGCTTTTTATTATATCAAGCCAGAGCCAGAGCATTTATTAAGTAA
CGCCAGATTTTTATATATAGTAGAGATAGAGCCAGAGGCGTTATATAACATTACCAGAGA
TATTAAGGGCTATTTAAAGAGCCAGAGCATAGATAGAGCATTACGCCAGATAAGGCGCAT
ATATAAAGGCGTAATTTATAAGATAGGTAATAAAGAGATAGTAAATTTATTTTATGATGT
TAATTTTATAAAAAGAGAGGTTTTAAGATGACCAGATTAGAATTTATAGAGCAATACCAG
AATAAAATAAATAAGATATTACCAGAATTTAAGAGCCAGCACTATATAGATAGTGATAAA
ATTGGTTGTTTATTTTTAGATAAATGGTATTATTACGATATAGATAAGTGCATAAAAATT
ATAGATATAACTAAAGAGGCAAGGGCTTATATTTATAATCTTAAAATAGGTAATAAGATT
AAAATAAATTCTTATGGTAATACGCCAGATGTAGAGGTTATAAAAATTAGTTACAAGAGG
CAATTAGATAAAAGAGGCAATAAAATAATCTCTACAATTACCACTAAAGAGATTTTTAGT
AATGGTGGTATATACGATAGCAAGGTAGATATTAAAGATATTATTTATAAAAGCAATACA
TATAGGGATATTTATGATAGTTATTTACATACAAACGGTTATTTTTTAAATTTTAGTAGT
TATGATATAGATTTAGCGGATAAAGTTATAGTCAATTACGAGGGCGTAAAATGAAAAATT
TAAAAAGATTTTAAAGAGGTTTGATTATGATAAAATTAAAAATAAATGATAAAGTAAAAA
TTAAATATACTGATAAATTTACTCACCCTTGCAGGATTATTTTTAGTAGTAAAATAGGTA
AAATAATTAAAATAAATAAAGGTATATACTTTATTAAATTTAAAGATAAAAATTTAATAG
GTTACGATTATGGTATAGGAACTTTTTATAAAAAAGATTTATTAAAAAGGAAATTTTAAA
ATGAGAGGAGAAACTAAATTTATAATCACTAAAGACCTGAAACATTTTAAAAGTGATATA
TTACACCATTGGGAATTAGCCAGAGCCAGAGGATATAAGTCAGAGGATATTTTAGAGAGT
GGATTATTTTTAGGTAATCAATTATTTATTTTAGAGAGCCAAGACCAGAGGCATTTAATA
AAACATTTAAGACATTATATAGGTAATATATTAGACCAATTAAATTCTAATTCTTTTGAT
ATAAGATTAAGGAATTGGTTAAAAGCCAGAGAATTAGAGAGCAGACTTTATTATTCTAAA
AGACCTTTGGGTATATTAAGAGAGGGCGACTAAATGATATATATAAGAGATAAAAAGTTA
TATAAAAATTTATCAGATTATTTAAAAGACGACTATAAAATAGTAAATTATAGAGTTAAG
GCATTTTATAGAGTATTAGAGCATTTATTAAATAAAGAGCCACTAAAAAATTACCCCGAT
ATTGATATTTATTTAGAAACTAAAAACGCCAGACCATTTGGGCGATTTACTACCAAGCCG
AGAAGTTTTTATCATAGGTATAATAAAGACCCTTATATTGTATTATATTTATCTACGCAT
TTATTAATTAAAAGCCAAGTTAAAAGTTTTATAGATACTATTTTACACGAGTTTATACAT
TATAATCAATGGTTAAATAATTTTAGGCAAGGGCATTATGGAATTTTAAAAGCCAATACC
AATGGCGACATAAATAAGTATATAGAGCAGTTAAAAATAAAATGATTAAAAAATTATTTT
TTAGAAATCTTGTAAGTTATTATAAAATAACGAGTTATAACAATAAAATAAACTTGACAA
GACCCGAAAAATATGTTATACTTAATTAGAAAGGATAAAAAGAGATGTATTTAAAATTAA
AAGAAATTAAATCAAAAGTAAAATTTTTAGGTAGTATAATTGATTTAGAAAGAGATAATA
AATTTAGATTATTAGTTGCGACTAATGGACATAGTTATTTAATTAGACCAGAATTAACTA
AAGATTTAGAAATAGATAAATGGTATAAATTTGAAGTAGATAATTCGGGCAATGTAATTA
TTAAAATAGAAAAATAAAATGACCGAGAGAATAAAAAGAAAGTGAGGTGATATAATGTTA
GAAAGATACAGGCAGAATTTAAAAGTAGTTGGTAATAAAGTATATTCT'''
(s.count('G') + s.count('C')) / len(s)


0.2390131071703932

In [ ]:
len(s)

3891